In [1]:
%load_ext pycodestyle_magic

%pycodestyle_on or %flake8_on

In [2]:
import numpy as np
import math


def PV_cal(F, C, n, m, r):
    return (C*(1-((1+r/m)**(-n)))/(r/m)+F/((1+r/m)**(n)))

In [19]:
# First we calculate YTM

Price = input('Current Bond Price:')
while True:
    try:
        Price = float(Price)
        if Price < 0:
            Price = input('Wrong input. Current Bond Price:')
            continue
        break
    except ValueError:
        Price = input('Wrong input. Current Bond Price:')

Par = input('Bond Par Value:')
while True:
    try:
        Par = float(Par)
        if Par < 0:
            Par = input('Wrong input. Bond Par Value:')
            continue
        break
    except ValueError:
        Par = input('Wrong input. Bond Par Value:')

CouponRate = input('Bond Coupon Rate (% p.a.):')
while True:
    try:
        CouponRate = float(CouponRate)/100
        break
    except ValueError:
        CouponRate = input('Wrong input. Bond Coupon Rate (% p.a.):')

Year = input('Years to Maturity:')
while True:
    try:
        Year = int(Year)
        if Year < 0:
            Year = input('Wrong input. Years to Maturity:')
            continue
        break
    except ValueError:
        Year = input('Wrong input. Years to Maturity:')

PaymentCnt = input('Number of Payments per Year:')
while True:
    try:
        PaymentCnt = int(PaymentCnt)
        if PaymentCnt < 0:
            PaymentCnt = input('Wrong input. Number of Payments per Year:')
            continue
        break
    except ValueError:
        PaymentCnt = input('Wrong input.Number of Payments per Year:')

coupon = Par * CouponRate / PaymentCnt
ytm = CouponRate+1e-8


while 1:
    pv = PV_cal(Par, coupon, PaymentCnt*Year, PaymentCnt, ytm)
    if abs(Price - pv) <= 0.0001:
        break
    ytm += 0.0000001

print('Yield to Maturity (YTM):', str(math.ceil(ytm * 1000000)/10000)+'%')
ans = [
        ['YTM', str(math.ceil(ytm * 1000000)/10000)+'%']
        ]

Current Bond Price: 950
Bond Par Value: 1000
Bond Coupon Rate (% p.a.): 0
Years to Maturity: 1
Number of Payments per Year: 1


Yield to Maturity (YTM): 5.2632%


In [4]:
payment_p = [math.ceil(principal/period)]
table1.append(['', '平均每月攤還本金', str(format(payment_p[0], ','))+'元'])
table1.append(['', '平均每月攤還利息', '請參考下表'])

payment_i = []
payment_total = []
for t in range(period):
    if payment_p[t] > principal-payment_p[t]:
        payment_p.append(int(principal-payment_p[t]))
    else:
        payment_p.append(payment_p[0])
    payment_i.append(round(principal * interest / 12))
    principal -= payment_p[t]
    if t == 0:
        payment_total.append(payment_p[t]+payment_i[t])
    else:
        payment_total.append(payment_total[t-1]+payment_p[t]+payment_i[t])
table1.append(['','全部利息', str(format(sum(payment_i),','))+'元'])
table1.append([])
table1.append(['','本金(元)','利息(元)','本金利息累計(元)'])

NameError: name 'principal' is not defined

18:18: E231 missing whitespace after ','
18:52: E231 missing whitespace after ','
20:18: E231 missing whitespace after ','
20:26: E231 missing whitespace after ','
20:34: E231 missing whitespace after ','


In [ ]:
for t in range(period):
    table1.append(['第'+str(t+1)+'期',format(payment_p[t],','),format(payment_i[t],','),format(payment_total[t],',')])
import csv
with open('本金平均攤還試算利息.csv', 'w', newline='',encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(table1)
